In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import numpy as np
from PIL import Image
import os

# Define the custom Fourier Transform layer
class FourierTransformLayer(layers.Layer):
    def call(self, inputs):
        # Apply 2D FFT
        fft = tf.signal.fft2d(tf.cast(inputs, tf.complex64))
        magnitude = tf.abs(fft)
        phase = tf.math.angle(fft)
        # Concatenate magnitude and phase along the channel dimension
        return tf.concat([magnitude, phase], axis=-1)

# Define the generator function to yield image and density map
def data_generator(image_dir, gt_dir):
    images = os.listdir(image_dir)
    for img_name in images:
        img_path = os.path.join(image_dir, img_name)
        gt_name = img_name.replace('IMG', 'GT_IMG').replace('.jpg', '.npy')  # Adjust for GT format
        gt_path = os.path.join(gt_dir, gt_name)

        # Load and resize image to 512x512
        image = Image.open(img_path).convert('RGB').resize((512, 512))
        image = np.array(image) / 255.0  # Normalize image to [0, 1]

        # Load and resize density map to 128x128 to match model output
        density_map = np.load(gt_path)
        density_map = np.array(Image.fromarray(density_map).resize((128, 128), Image.BILINEAR))
        
        yield image.astype(np.float32), density_map[..., np.newaxis].astype(np.float32)

# Define a function to wrap the generator with tf.data.Dataset
def create_dataset(image_dir, gt_dir, batch_size=8):
    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(image_dir, gt_dir),
        output_signature=(
            tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float32),
        )
    )
    dataset = dataset.batch(batch_size)
    return dataset

# Create Spatial Path of Dual-Path Network
def create_spatial_path():
    inputs = layers.Input(shape=(512, 512, 3))
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = layers.MaxPooling2D((2, 2))(x)
    
    # ASPP Module
    aspp1 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=1, activation='relu')(x)
    aspp2 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=2, activation='relu')(x)
    aspp3 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=3, activation='relu')(x)
    
    aspp_concat = layers.Concatenate()([aspp1, aspp2, aspp3])
    aspp_output = layers.Conv2D(512, (1, 1), activation='relu')(aspp_concat)

    # Decoder
    x = layers.UpSampling2D((2, 2))(aspp_output)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    
    return models.Model(inputs, x)

# Create Frequency Path of Dual-Path Network
def create_frequency_path():
    inputs = layers.Input(shape=(512, 512, 3))

    # Apply the custom Fourier transform layer
    freq_input = FourierTransformLayer()(inputs)
    
    x = layers.Conv2D(64, (3, 3), strides=2, padding='same', activation='relu')(freq_input)
    x = layers.Conv2D(128, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Reshape((1, 1, 256))(x)
    
    return models.Model(inputs, x)

# Fusion and final prediction layers
def create_dual_path_network():
    spatial_path = create_spatial_path()
    frequency_path = create_frequency_path()

    inputs = layers.Input(shape=(512, 512, 3))
    spatial_output = spatial_path(inputs)
    frequency_output = frequency_path(inputs)
    
    # Upsample frequency output to match spatial dimensions
    frequency_upsampled = layers.UpSampling2D((128, 128))(frequency_output)

    # Fusion layer
    x = layers.Concatenate()([spatial_output, frequency_upsampled])
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(1, (1, 1), activation='relu')(x)

    return models.Model(inputs, x)

# Create the model
model = create_dual_path_network()
model.summary()

# Compile the model with L1 loss
model.compile(optimizer='adam', loss='mean_absolute_error')

# Paths for training and testing data
train_image_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\train_data\\images'
train_gt_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\train_data\\ground_truth_npy'
test_image_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\test_data\\images'
test_gt_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\test_data\\ground_truth_npy'

# Create train and test datasets
train_dataset = create_dataset(train_image_dir, train_gt_dir, batch_size=8)
test_dataset = create_dataset(test_image_dir, test_gt_dir, batch_size=8)

# Train the model
model.fit(train_dataset, epochs=10)

# Evaluate on the test set to compute MSE and MAE
predictions = model.predict(test_dataset)

# Collect all predictions and ground truth to compute MSE and MAE
all_preds, all_gts = [], []
for image_batch, gt_batch in test_dataset:
    pred_batch = model.predict(image_batch)
    all_preds.append(pred_batch)
    
    
    all_gts.append(gt_batch)

# Concatenate all batches to form arrays
all_preds = np.concatenate(all_preds, axis=0)
all_gts = np.concatenate(all_gts, axis=0)

# Calculate Mean Squared Error (MSE) and Mean Absolute Error (MAE)
mse = np.mean((all_preds - all_gts) ** 2)
mae = np.mean(np.abs(all_preds - all_gts))

print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")


C:\Users\hp\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 512, 512, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ functional_1 (Functional)     │ (None, 1, 1, 256)         │         372,544 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ functional (Functional)       │ (None, 128, 128, 128)     │      10,093,760 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_2               │ (None, 128, 128, 256)     │               0 │ functional_1[0][0]         │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 128, 128, 384)     │               0 │ functional[0][0],          │
│                               │                           │                 │ up_sampling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_17 (Conv2D)            │ (None, 128, 128, 128)     │         442,496 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_18 (Conv2D)            │ (None, 128, 128, 64)      │          73,792 │ conv2d_17[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_19 (Conv2D)            │ (None, 128, 128, 1)       │              65 │ conv2d_18[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 10,982,657 (41.90 MB)

 Trainable params: 10,982,657 (41.90 MB)

...

Epoch 1/10


C:\Users\hp\AppData\Local\Temp\ipykernel_2516\3187452816.py:31: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  density_map = np.array(Image.fromarray(density_map).resize((128, 128), Image.BILINEAR))


38/38 ━━━━━━━━━━━━━━━━━━━━ 1913s 50s/step - loss: 488.3505


C:\Users\hp\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


Epoch 2/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1836s 48s/step - loss: 186.0190
Epoch 3/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1879s 49s/step - loss: 188.9505
Epoch 4/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1861s 49s/step - loss: 185.4199
Epoch 5/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1839s 48s/step - loss: 184.1398
Epoch 6/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1660s 44s/step - loss: 182.2210
Epoch 7/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1636s 43s/step - loss: 181.4058
Epoch 8/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1648s 43s/step - loss: 180.0414
Epoch 9/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1637s 43s/step - loss: 179.4263
Epoch 10/10
38/38 ━━━━━━━━━━━━━━━━━━━━ 1623s 43s/step - loss: 178.5081
23/23 ━━━━━━━━━━━━━━━━━━━━ 233s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 10s 10s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step
1/

In [ ]:

import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

# Define the custom Fourier Transform layer
class FourierTransformLayer(layers.Layer):
    def call(self, inputs):
        # Apply 2D FFT
        fft = tf.signal.fft2d(tf.cast(inputs, tf.complex64))
        magnitude = tf.abs(fft)
        phase = tf.math.angle(fft)
        # Concatenate magnitude and phase along the channel dimension
        return tf.concat([magnitude, phase], axis=-1)

# Define the generator function to yield image and density map
def data_generator(image_dir, gt_dir):
    images = os.listdir(image_dir)
    for img_name in images:
        img_path = os.path.join(image_dir, img_name)
        gt_name = img_name.replace('IMG', 'GT_IMG').replace('.jpg', '.npy')  # Adjust for GT format
        gt_path = os.path.join(gt_dir, gt_name)

        # Load and resize image to 512x512
        image = Image.open(img_path).convert('RGB').resize((512, 512))
        image = np.array(image) / 255.0  # Normalize image to [0, 1]

        # Load and resize density map to 128x128 to match model output
        density_map = np.load(gt_path)
        density_map = np.array(Image.fromarray(density_map).resize((128, 128), Image.BILINEAR))
        
        yield image.astype(np.float32), density_map[..., np.newaxis].astype(np.float32)

# Define a function to wrap the generator with tf.data.Dataset
def create_dataset(image_dir, gt_dir, batch_size=8):
    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(image_dir, gt_dir),
        output_signature=(
            tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float32),
        )
    )
    dataset = dataset.batch(batch_size)
    return dataset

# Create Spatial Path of Dual-Path Network
def create_spatial_path():
    inputs = layers.Input(shape=(512, 512, 3))
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # ASPP Module with multiple dilation rates
    aspp1 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=1, activation='relu')(x)
    aspp2 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=6, activation='relu')(x)
    aspp3 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=12, activation='relu')(x)
    aspp4 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=18, activation='relu')(x)

    aspp_concat = layers.Concatenate()([aspp1, aspp2, aspp3, aspp4])
    aspp_output = layers.Conv2D(512, (1, 1), activation='relu')(aspp_concat)

    # Decoder
    x = layers.UpSampling2D((2, 2))(aspp_output)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    
    return models.Model(inputs, x)

# Create Frequency Path of Dual-Path Network
def create_frequency_path():
    inputs = layers.Input(shape=(512, 512, 3))

    # Apply the custom Fourier transform layer
    freq_input = FourierTransformLayer()(inputs)
    
    x = layers.Conv2D(64, (3, 3), strides=2, padding='same', activation='relu')(freq_input)
    x = layers.Conv2D(128, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Reshape((1, 1, 256))(x)
    
    return models.Model(inputs, x)

# Fusion and final prediction layers
def create_dual_path_network():
    spatial_path = create_spatial_path()
    frequency_path = create_frequency_path()

    inputs = layers.Input(shape=(512, 512, 3))
    spatial_output = spatial_path(inputs)
    frequency_output = frequency_path(inputs)
    
    # Upsample frequency output to match spatial dimensions
    frequency_upsampled = layers.UpSampling2D((128, 128))(frequency_output)

    # Fusion layer
    x = layers.Concatenate()([spatial_output, frequency_upsampled])
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(1, (1, 1), activation='relu')(x)

    return models.Model(inputs, x)

# Create the model
model = create_dual_path_network()
model.summary()

# Compile the model with a lower learning rate
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_absolute_error')

# Paths for training and testing data
train_image_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\train_data\\images'
train_gt_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\train_data\\ground_truth_npy'
test_image_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\test_data\\images'
test_gt_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\test_data\\ground_truth_npy'

# Create train and test datasets
train_dataset = create_dataset(train_image_dir, train_gt_dir, batch_size=8)
test_dataset = create_dataset(test_image_dir, test_gt_dir, batch_size=8)

# Train the model for more epochs
model.fit(train_dataset, epochs=50, validation_data=test_dataset)

# Evaluate on the test set to compute MSE and MAE
all_preds, all_gts = [], []
for image_batch, gt_batch in test_dataset:
    pred_batch = model.predict(image_batch)
    all_preds.append(pred_batch)
    all_gts.append(gt_batch)

# Concatenate all batches to form arrays
all_preds = np.concatenate(all_preds, axis=0)
all_gts = np.concatenate(all_gts, axis=0)

# Calculate Mean Squared Error (MSE) and Mean Absolute Error (MAE)
mse = np.mean((all_preds - all_gts) ** 2)
mae = np.mean(np.abs(all_preds - all_gts))

print(f"Test MSE: {mse:.4f}")
print(f"Test MAE: {mae:.4f}")


C:\Users\hp\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)    │ (None, 512, 512, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ functional_1 (Functional)     │ (None, 1, 1, 256)         │         372,544 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ functional (Functional)       │ (None, 128, 128, 128)     │      11,412,416 │ input_layer_2[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ up_sampling2d_2               │ (None, 128, 128, 256)     │               0 │ functional_1[0][0]         │
│ (UpSampling2D)                │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ concatenate_1 (Concatenate)   │ (None, 128, 128, 384)     │               0 │ functional[0][0],          │
│                               │                           │                 │ up_sampling2d_2[0][0]      │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_18 (Conv2D)            │ (None, 128, 128, 128)     │         442,496 │ concatenate_1[0][0]        │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_19 (Conv2D)            │ (None, 128, 128, 64)      │          73,792 │ conv2d_18[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2d_20 (Conv2D)            │ (None, 128, 128, 1)       │              65 │ conv2d_19[0][0]            │
└───────────────────────────────┴───────────────────────────┴─────────────────┴────────────────────────────┘

 Total params: 12,301,313 (46.93 MB)

 Trainable params: 12,297,473 (46.91 MB)

...

Epoch 1/50


C:\Users\hp\AppData\Local\Temp\ipykernel_19376\1745612280.py:33: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  density_map = np.array(Image.fromarray(density_map).resize((128, 128), Image.BILINEAR))


     38/Unknown 3813s 100s/step - loss: 282.6423

C:\Users\hp\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


38/38 ━━━━━━━━━━━━━━━━━━━━ 4120s 109s/step - loss: 281.3203 - val_loss: 261.0369
Epoch 2/50
38/38 ━━━━━━━━━━━━━━━━━━━━ 4350s 114s/step - loss: 172.8208 - val_loss: 243.2196
Epoch 3/50
14/38 ━━━━━━━━━━━━━━━━━━━━ 43:19 108s/step - loss: 176.6655

In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.layers import BatchNormalization, Dropout
from tensorflow.keras.optimizers import Adam
import numpy as np
from PIL import Image
import os

# Define the custom Fourier Transform layer
class FourierTransformLayer(layers.Layer):
    def call(self, inputs):
        # Apply 2D FFT
        fft = tf.signal.fft2d(tf.cast(inputs, tf.complex64))
        magnitude = tf.abs(fft)
        phase = tf.math.angle(fft)
        # Concatenate magnitude and phase along the channel dimension
        return tf.concat([magnitude, phase], axis=-1)

# Define the generator function to yield image and density map
def data_generator(image_dir, gt_dir):
    images = os.listdir(image_dir)
    for img_name in images:
        img_path = os.path.join(image_dir, img_name)
        gt_name = img_name.replace('IMG', 'GT_IMG').replace('.jpg', '.npy')  # Adjust for GT format
        gt_path = os.path.join(gt_dir, gt_name)

        # Load and resize image to 512x512, use float32 for lower memory usage
        image = Image.open(img_path).convert('RGB').resize((512, 512))
        image = np.array(image, dtype=np.float32) / 255.0  # Normalize image to [0, 1]

        # Load and resize density map to 128x128 to match model output
        density_map = np.load(gt_path)
        density_map = np.array(Image.fromarray(density_map).resize((128, 128), Image.BILINEAR), dtype=np.float32)
        
        yield image, density_map[..., np.newaxis]
        
        
# Define a function to calculate and print scaled MSE and MAE
def calculate_and_print_metrics(predictions, ground_truths, mse_factor=1, mae_factor=1):
    mse = np.mean((predictions - ground_truths) ** 2) / mse_factor
    mae = np.mean(np.abs(predictions - ground_truths)) / mae_factor
    print(f"Test MSE: {mse:.4f}")
    print(f"Test MAE: {mae:.4f}")

# Define a function to wrap the generator with tf.data.Dataset
def create_dataset(image_dir, gt_dir, batch_size=4):  # Reduced batch size for lower memory
    dataset = tf.data.Dataset.from_generator(
        lambda: data_generator(image_dir, gt_dir),
        output_signature=(
            tf.TensorSpec(shape=(512, 512, 3), dtype=tf.float32),
            tf.TensorSpec(shape=(128, 128, 1), dtype=tf.float32),
        )
    )
    # Use prefetch to optimize CPU processing, batch the dataset
    dataset = dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    return dataset

# Create the Dual-Path Network (Spatial and Frequency paths defined as per original code)
def create_spatial_path():
    inputs = layers.Input(shape=(512, 512, 3))
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(inputs)
    x = BatchNormalization()(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)
    x = Dropout(0.3)(x)

    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.Conv2D(512, (3, 3), padding='same', activation='relu')(x)
    x = BatchNormalization()(x)
    x = layers.MaxPooling2D((2, 2))(x)

    # ASPP Module with multiple dilation rates
    aspp1 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=1, activation='relu')(x)
    aspp2 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=6, activation='relu')(x)
    aspp3 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=12, activation='relu')(x)
    aspp4 = layers.Conv2D(256, (3, 3), padding='same', dilation_rate=18, activation='relu')(x)

    aspp_concat = layers.Concatenate()([aspp1, aspp2, aspp3, aspp4])
    aspp_output = layers.Conv2D(512, (1, 1), activation='relu')(aspp_concat)

    # Decoder
    x = layers.UpSampling2D((2, 2))(aspp_output)
    x = layers.Conv2D(256, (3, 3), padding='same', activation='relu')(x)
    x = layers.UpSampling2D((2, 2))(x)
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    
    return models.Model(inputs, x)

def create_frequency_path():
    inputs = layers.Input(shape=(512, 512, 3))
    freq_input = FourierTransformLayer()(inputs)
    
    x = layers.Conv2D(64, (3, 3), strides=2, padding='same', activation='relu')(freq_input)
    x = layers.Conv2D(128, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.Conv2D(256, (3, 3), strides=2, padding='same', activation='relu')(x)
    x = layers.GlobalAveragePooling2D()(x)
    x = layers.Reshape((1, 1, 256))(x)
    
    return models.Model(inputs, x)

def create_dual_path_network():
    spatial_path = create_spatial_path()
    frequency_path = create_frequency_path()

    inputs = layers.Input(shape=(512, 512, 3))
    spatial_output = spatial_path(inputs)
    frequency_output = frequency_path(inputs)
    
    frequency_upsampled = layers.UpSampling2D((128, 128))(frequency_output)
    x = layers.Concatenate()([spatial_output, frequency_upsampled])
    x = layers.Conv2D(128, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(64, (3, 3), padding='same', activation='relu')(x)
    x = layers.Conv2D(1, (1, 1), activation='relu')(x)

    return models.Model(inputs, x)

model = create_dual_path_network()
model.compile(optimizer=Adam(learning_rate=0.0001), loss='mean_absolute_error')

# Paths for training and testing data
train_image_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\train_data\\images'
train_gt_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\train_data\\ground_truth_npy'
test_image_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\test_data\\images'
test_gt_dir = 'C:\\Users\\hp\\Downloads\\ShanghaiTech_Crowd_Counting_Dataset\\part_A_final\\test_data\\ground_truth_npy'

# Create train and test datasets with optimized batch size and memory handling
train_dataset = create_dataset(train_image_dir, train_gt_dir, batch_size=4)
test_dataset = create_dataset(test_image_dir, test_gt_dir, batch_size=4)

# Train the model
model.fit(train_dataset, epochs=10, validation_data=test_dataset)



# Evaluate model and calculate metrics
all_preds, all_gts = [], []
for image_batch, gt_batch in test_dataset:
    pred_batch = model.predict(image_batch)
    all_preds.append(pred_batch)
    all_gts.append(gt_batch)

all_preds = np.concatenate(all_preds, axis=0)
all_gts = np.concatenate(all_gts, axis=0)

calculate_and_print_metrics(all_preds, all_gts)


C:\Users\hp\anaconda3\lib\site-packages\scipy\__init__.py:155: UserWarning: A NumPy version >=1.18.5 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"



Epoch 1/10


C:\Users\hp\AppData\Local\Temp\ipykernel_16820\9274737.py:33: DeprecationWarning: BILINEAR is deprecated and will be removed in Pillow 10 (2023-07-01). Use Resampling.BILINEAR instead.
  density_map = np.array(Image.fromarray(density_map).resize((128, 128), Image.BILINEAR), dtype=np.float32)


     75/Unknown 2913s 39s/step - loss: 239.9554

C:\Users\hp\anaconda3\lib\contextlib.py:137: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self.gen.throw(typ, value, traceback)


75/75 ━━━━━━━━━━━━━━━━━━━━ 3152s 42s/step - loss: 239.3412 - val_loss: 255.5335
Epoch 2/10
 2/75 ━━━━━━━━━━━━━━━━━━━━ 50:59 42s/step - loss: 219.6402

KeyboardInterrupt: 